In [ ]:
y = df['quality']
X = df[["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=21)

numeric_features = ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"]
numeric_transformer = Pipeline([
    ('stdscaler', MinMaxScaler()),  
])

preprocessor = ColumnTransformer(
    remainder="passthrough",
    transformers=[
        ('num', numeric_transformer, numeric_features),
       
    ]
)

model = SVR(kernel="poly",C=3, gamma=1 )

pipe = Pipeline([
    ('prep', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)

In [ ]:
pipe.predict(X_test)
pipe.score(X_test, y_test)